In [26]:
import pandas as pd
import numpy as np

In [57]:
labels = ["O", "I-BRAND", "B-BRAND", "B-PRODUCT","I-PRODUCT", "I-PRICE","B-PRICE", "B-LOC", "I-LOC"]  
num_labels = len(labels)



In [58]:
import pandas as pd
labled_data=pd.read_csv("./data/datalabeled_messages_output.csv")
labled_data.head()

,Formatted_Labeled_Message
0,dell 0 I-BRAND\ng15 ...
1,asus 0 I-BRAND\nzeyph...
2,from 0 O\nneva ...
3,may 0 O\n2017 ...
4,from 0 O\nneva ...


In [59]:
def process_labeled_message(message):
    
    if not isinstance(message, str):
        return pd.DataFrame(columns=['Token', 'Position', 'Label'])
    
   
    lines = message.split('\n')
    
   
    tokens = []
    positions = []
    labels = []
    
    for line in lines:
      
        parts = line.split()
        if len(parts) == 3:  
            token = parts[0]
            position = parts[1]
            label = parts[2]
            
            tokens.append(token)
            positions.append(position)
            labels.append(label)
    
    return pd.DataFrame({'Token': tokens, 'Position': positions, 'Label': labels})


processed_dfs = labled_data['Formatted_Labeled_Message'].apply(process_labeled_message)


result_df = pd.concat(processed_dfs.values, ignore_index=True)


result_df.head()

,Token,Position,Label
0,dell,0,I-BRAND
1,g15,5,O
2,156,1,O
3,full,5,O
4,hd,10,O


In [60]:

sentence = ' '.join(result_df['Token'].tolist())
labels = ' '.join(result_df['Label'].tolist())


sentence_df = pd.DataFrame({'Sentence': [sentence], 'Labels': [labels]})

print(sentence_df)


                                            Sentence  \
0  dell g15 156 full hd 165hz core i7 11th genera...   

                                              Labels  
0  I-BRAND O O O O O O O O O O O O O O O O O O O ...  


In [61]:
label_list = result_df['Label'].explode().unique().tolist()  
label_to_id = {label: i for i, label in enumerate(label_list)}  
id_to_label = {i: label for label, i in label_to_id.items()}  
print(label_to_id)

{'I-BRAND': 0, 'O': 1, 'I-PRICE': 2, 'I-LOC': 3, 'B-PRODUCT': 4}


In [62]:
from transformers import XLMRobertaTokenizer
from datasets import Dataset

# Load the tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

# Convert the sentence DataFrame to a Hugging Face Dataset
huggingface_dataset = Dataset.from_pandas(sentence_df)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['Sentence'], truncation=True, padding='max_length', return_tensors="pt")

    # Initialize labels for tokenized inputs
    labels = []
    
    for i, label_sequence in enumerate(examples['Labels']):
        label_ids = []
        label_sequence = label_sequence.split()
        
        # Create a mapping from token to label
        label_ids_map = {word: idx for idx, word in enumerate(label_sequence)}

        # Align labels with tokens
        for token in tokenizer.convert_ids_to_tokens(tokenized_inputs['input_ids'][0].tolist()):
            if token in label_ids_map:
                label_ids.append(label_ids_map[token])  
            else:
                label_ids.append(-100)  
            
        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs

# Apply the tokenization function to the dataset
tokenized_dataset = huggingface_dataset.map(tokenize_and_align_labels, batched=True)

# Check the tokenized dataset
print(tokenized_dataset)


c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(




Map: 100%|██████████| 1/1 [00:02<00:00,  2.31s/ examples]

Dataset({
    features: ['Sentence', 'Labels', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1
})


In [65]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./newresults',          # Output directory
    evaluation_strategy="epoch",     # Evaluation strategy
     learning_rate=1e-5,              # Learning rate
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=64,   # Batch size for evaluation
    num_train_epochs=3,              # Number of training epochs
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./newlogs',            # Directory for logs
    logging_steps=10,                # Log every 10 steps
)


In [66]:
from transformers import XLMRobertaForTokenClassification, Trainer

# Load the pre-trained model
model = XLMRobertaForTokenClassification.from_pretrained('xlm-roberta-base', num_labels=len(set(result_df['Label'])))

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset, 
)

# Train the model
trainer.train()


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.












                               



                                 




                                     
                                  


                                    







  0%|          | 0/3 [30:39<?, ?it/s]







{'eval_loss': nan, 'eval_runtime': 3.5272, 'eval_samples_per_second': 0.284, 'eval_steps_per_second': 0.284, 'epoch': 1.0}











                               



                                             
                                  


                                    




                      



  0%|          | 0/3 [31:14<?, ?it/s]











{'eval_loss': nan, 'eval_runtime': 2.3937, 'eval_samples_per_second': 0.418, 'eval_steps_per_second': 0.418, 'epoch': 2.0}


SafetensorError: Error while serializing: IoError(Os { code: 112, kind: StorageFull, message: "There is not enough space on the disk." })

In [67]:
from transformers import DistilBertForTokenClassification, Trainer

# Load the pre-trained DistilBERT model
distilbert_model = DistilBertForTokenClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=len(set(result_df['Label'])))

# Define the Trainer for DistilBERT
distilbert_trainer = Trainer(
    model=distilbert_model,
    args=training_args,  # Same training arguments used earlier
    train_dataset=tokenized_dataset,  # Same tokenized dataset
    eval_dataset=tokenized_dataset, 
)

# Train the DistilBERT model
distilbert_trainer.train()


c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:983: UserWarning: Not enough free disk space to download the file. The expected file size is: 541.80 MB. The target location C:\Users\user\.cache\huggingface\hub\models--distilbert-base-multilingual-cased\blobs only has 288.38 MB free disk space.
  warnings.warn(
Error while downloading from https://cdn-lfs.hf.co/distilbert-base-multilingual-cased/fcf002be901b9ad708e0df430b8d18deaa4b3fe3519e24e9017a5ee17ca2c228?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1728154478&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyODE1NDQ3OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9kaXN0aWxiZXJ0LWJhc2UtbXVsdGlsaW5ndWFsLWNhc2VkL2ZjZjAwMmJlOTAxYjlhZDcwOGUwZGY0MzBiOGQxOGRlYWE0YjNmZTM1MTllMjRlOTAxN2E1ZWUxN2NhMmMyMjg%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signat

ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'cdn-lfs.hf.co\', port=443): Max retries exceeded with url: /distilbert-base-multilingual-cased/fcf002be901b9ad708e0df430b8d18deaa4b3fe3519e24e9017a5ee17ca2c228?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1728154478&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyODE1NDQ3OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9kaXN0aWxiZXJ0LWJhc2UtbXVsdGlsaW5ndWFsLWNhc2VkL2ZjZjAwMmJlOTAxYjlhZDcwOGUwZGY0MzBiOGQxOGRlYWE0YjNmZTM1MTllMjRlOTAxN2E1ZWUxN2NhMmMyMjg~cmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=rBqxxHksKw~5OjnJ78gWtfJK6wWCAtJvmXQQxFFtDReKS0WJ8q328Y6x8N~O3fjd60usQzoib847h57wnJkgEpaT7QValwPXwpvL9~rZssDzjiK8zjRzXf5Uh044EsvmntgWF~FtOWXvUyeUFMPR0jHxK3ojuByCxAV9jGcw0BUrPjzA8r652shyX03pX6BSQTgqw4aa5O5-LwWRPtsJ5eq-3L5NcRYYnG-bFtQBOvMLKhRgQQjMad7rgRbGamp0JRTXYMLbo9Xw-TFZGamr3drUa3W003DPC0~7mUa4wW

In [68]:
from transformers import BertForTokenClassification, Trainer

# Load the pre-trained mBERT model
mbert_model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(set(result_df['Label'])))

# Define the Trainer for mBERT
mbert_trainer = Trainer(
    model=mbert_model,
    args=training_args,  # Same training arguments used earlier
    train_dataset=tokenized_dataset,  # Same tokenized dataset
    eval_dataset=tokenized_dataset, 
)

# Train the mBERT model
mbert_trainer.train()


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 3/3 [21:51<00:00, 437.21s/it]




IndexError: index out of range in self

In [69]:
# Evaluate XLM-Roberta
xlm_roberta_eval_results = trainer.evaluate()
print(f"XLM-Roberta evaluation results: {xlm_roberta_eval_results}")

# Evaluate DistilBERT
distilbert_eval_results = distilbert_trainer.evaluate()
print(f"DistilBERT evaluation results: {distilbert_eval_results}")

# Evaluate mBERT
mbert_eval_results = mbert_trainer.evaluate()
print(f"mBERT evaluation results: {mbert_eval_results}")








                               



                                 


                            




                                     
                                  









  0%|          | 0/3 [44:15<?, ?it/s]





{'eval_loss': nan, 'eval_runtime': 2.0186, 'eval_samples_per_second': 0.495, 'eval_steps_per_second': 0.495, 'epoch': 3.0}
XLM-Roberta evaluation results: {'eval_loss': nan, 'eval_runtime': 2.0186, 'eval_samples_per_second': 0.495, 'eval_steps_per_second': 0.495, 'epoch': 3.0}


NameError: name 'distilbert_trainer' is not defined